# Figure 5c Data Generation

This notebook processes raw driven spin coherence experimental data to generate processed data for Figure 5c.

## Process:
1. Load raw experimental data from UUIDs
2. Process data (fit decaying sinusoids, extract frequencies and T2 values)
3. Save processed data for plotting





In [1]:
# Import common utilities
import sys
from pathlib import Path

# Add common directory to path (works in Jupyter notebooks)
common_path = Path().resolve().parent / 'common_scripts'
sys.path.insert(0, str(common_path))

from raw_data_loader import load_raw_data_by_uuid
from data_saver import save_figure_data
from data_loader import load_figure_data
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import linregress

# Data path

data_path = Path().resolve().parent / 'data'
# Data path

# Alias for convenience
load_by_uuid = load_raw_data_by_uuid

# Function to fit decaying sinusoid
def decaying_sinusoid(t, A, f, phi, T2, offset):
    """Fit function for decaying sinusoid."""
    return A * np.exp(-t/T2) * np.sin(2*np.pi*f*t + phi) + offset




This section: loads raw data, processes it, and saves to files


In [2]:
# Define amplitudes and UUIDs for different delay times (cv_stop values)
# cv_stop represents different delay times: 0ns, 5ns, 10ns, 15ns, 19ns
amplitudes_for0ns = np.linspace(1000,2500,15)
cvstop_0ns = [1728762628691108893,
17287_62235_433108893,
17287_61837_491108893,
17287_61449_358108893,
17287_61060_213108893,
17287_60677_728108893,
17287_60291_145108893,
17287_59895_996108893,
17287_59508_289108893,
17287_59125_972108893,
17287_58729_080108893,
17287_58343_802108893,
17287_57951_293108893,
17287_57558_319108893,
17287_57155_548108893]

amplitudes_for5ns = np.linspace(1000,2500,15)
cvstop_5ns = [1728772391997108893,
17287_71799_045108893,
17287_71210_910108893,
17287_70619_106108893,
17287_70041_580108893,
17287_69464_233108893,
17287_68887_800108893,
17287_68296_194108893,
17287_68296_194108893,
17287_67137_637108893,
17287_66563_069108893,
17287_65996_056108893,
17287_65419_147108893,
17287_64839_393108893,
17287_64244_589108893
]

amplitudes_for10ns = np.linspace(1000,2500,20)
cvstop_10ns = [1728687246829108893,17286_86179_746108893,17286_85139_344108893,
17286_84102_042108893,17286_83077_742108893,17286_82040_663108893,
17286_80998_493108893,17286_79958_316108893,17286_78928_485108893,
17286_77858_274108893,17286_76823_208108893,17286_75777_993108893,
17286_74774_656108893,17286_73756_854108893,17286_72698_492108893,
17286_71642_506108893,17286_70627_766108893,17286_69602_906108893,
17286_68607_693108893,17286_67575_447108893
]

amplitudes_for15ns = np.linspace(1000,2500,15)
cvstop_15ns = [1728752414376108893,
17287_51028_254108893,
17287_49629_878108893,
17287_48299_306108893,
17287_46771_366108893,
17287_45417_265108893,
17287_43962_702108893,
17287_42620_811108893,
17287_41304_272108893,
17287_39874_643108893,
17287_38435_372108893,
17287_37105_343108893,
17287_35801_633108893,
17287_34464_966108893,
17287_33105_347108893]

amplitudes_for19ns = np.linspace(1000,2500,15)
cvstop_19ns = [1728867344002108893,
17288_63421_452108893,
17288_59537_184108893,
17288_55548_986108893,
17288_51722_210108893,
17288_48791_839108893,
17288_45003_907108893,
17288_42159_200108893,
17288_38314_044108893,
17288_34398_588108893,
17288_30497_710108893,
17288_26565_248108893,
17288_22744_190108893,
17288_19301_035108893,
17288_15778_175108893]
# Combine all datasets
cv_stops = [0, 5, 10, 15, 19]
all_datasets = [cvstop_0ns, cvstop_5ns, cvstop_10ns, cvstop_15ns, cvstop_19ns]
all_amplitudes = [
    np.flip(amplitudes_for0ns), np.flip(amplitudes_for5ns),
    np.flip(amplitudes_for10ns), np.flip(amplitudes_for15ns),
    np.flip(amplitudes_for19ns)
]

# Load all high field datasets
print("Loading driven spin coherence datasets...")
loaded_datasets = []
for dataset_uuids in all_datasets:
    dataset_group = []
    for uuid in dataset_uuids:
        try:
            uuid = int(str(uuid).replace('_', ''))
            ds = load_by_uuid(uuid, data_path)
            dataset_group.append(ds)
        except Exception as e:
            dataset_group.append(None)
    loaded_datasets.append(dataset_group)
    print(f"✓ Loaded {sum(ds is not None for ds in dataset_group)}/{len(dataset_group)} datasets for cv_stop={cv_stops[len(loaded_datasets)-1]}")

# Load low field datasets
cv_stop_0ns_lowfield = [1730906893566108893, 1730933545873108893, 1730935987842108893, 1730938073783108893]
print("\nLoading low field datasets...")
lowfield_datasets = []
for uuid in cv_stop_0ns_lowfield:
    try:
        ds = load_by_uuid(uuid, data_path)
        lowfield_datasets.append(ds)
    except Exception as e:
        lowfield_datasets.append(None)
print(f"✓ Loaded {sum(ds is not None for ds in lowfield_datasets)}/{len(cv_stop_0ns_lowfield)} low field datasets")


Loading driven spin coherence datasets...
✓ Loaded 15/15 datasets for cv_stop=0
✓ Loaded 15/15 datasets for cv_stop=5
✓ Loaded 20/20 datasets for cv_stop=10
✓ Loaded 15/15 datasets for cv_stop=15
✓ Loaded 15/15 datasets for cv_stop=19

Loading low field datasets...
✓ Loaded 4/4 low field datasets


## Process High Field Data


In [3]:
# Process high field data - fit decaying sinusoids
all_frequencies = []
all_T2_values = []
all_cv_stops = []
all_dT2_values = []

for cv_stop, datasets, amplitudes in zip(cv_stops, loaded_datasets, all_amplitudes):
    for ds, amplitude in zip(datasets, amplitudes):
        if ds is None:
            continue
            
        try:
            data = ds.m1_5()  # Get first measurement
            data = data[~np.isnan(data)]  # Remove NaN values
            time_points = ds.m1_5.x() * (20 - cv_stop) * 2  # Convert to ns
            time_points = time_points[~np.isnan(data)]  # Match time points to filtered data
            
            if len(data) < 10:  # Skip if too few points
                continue
            
            # Initial parameter guesses
            A_guess = (np.max(data) - np.min(data)) / 2
            f_guess = 0.001  # Initial frequency guess in GHz
            T2_guess = np.max(time_points)
            offset_guess = np.mean(data)
            
            # Fit the data
            popt, pcov = curve_fit(decaying_sinusoid, time_points, data,
                                 p0=[A_guess, f_guess, 0, T2_guess, offset_guess],
                                 maxfev=5000)
            
            # Calculate errors from covariance matrix
            perr = np.sqrt(np.diag(pcov))
            
            # Only include results if relative errors are below 50%
            if (perr[1]/abs(popt[1]) < 0.5) and (perr[3]/abs(popt[3]) < 0.5):
                all_frequencies.append(popt[1])  # Store frequency
                all_T2_values.append(popt[3])  # Store T2
                all_dT2_values.append(perr[3])  # Store T2 error
                all_cv_stops.append(cv_stop)

        except Exception as e:
            continue

print(f"Processed {len(all_frequencies)} high field data points")


Processed 74 high field data points


/var/folders/wj/x78z9z710q7csr068fhd10bxk7mx3x/T/ipykernel_84186/3117269083.py:28: RuntimeWarning: overflow encountered in exp
  return A * np.exp(-t/T2) * np.sin(2*np.pi*f*t + phi) + offset
/var/folders/wj/x78z9z710q7csr068fhd10bxk7mx3x/T/ipykernel_84186/28282803.py:28: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(decaying_sinusoid, time_points, data,


## Process Low Field Data


In [4]:
# Process low field data
def analyze_rabi_frequency(ds, is_lowfield=True):
    """Analyze Rabi oscillations from a dataset"""
    if ds is None:
        return None, None, None, None
        
    try:
        # Get measurement data based on field condition
        data = ds.m1_3() if is_lowfield else ds.m1_5()
        if len(data.shape) > 1:
            data = data.reshape(-1)
            
        # Remove NaN/inf values
        valid_mask = ~(np.isnan(data) | np.isinf(data))
        data = data[valid_mask]
        
        # Get time points
        if is_lowfield:
            shuttle_round = ds.m1_3.x()[valid_mask]
            time = shuttle_round * 5.4 * 2  # ns
        else:
            time = ds.m1_5.x()[valid_mask] * 20 * 2  # ns
        
        # Initial parameter guesses
        A_guess = (np.max(data) - np.min(data))/2
        f_guess = 0.001  # GHz
        tau_guess = np.max(time)
        offset_guess = np.mean(data)
        
        # Fit data
        popt, pcov = curve_fit(decaying_sinusoid, time, data,
                           p0=[A_guess, f_guess, 0, tau_guess, offset_guess],
                           maxfev=5000)
        
        # Extract parameters and errors
        f_fit, tau_fit = popt[1], popt[3]
        perr = np.sqrt(np.diag(pcov))
        f_err, tau_err = perr[1], perr[3]
        
        return f_fit, tau_fit, f_err, tau_err
        
    except Exception as e:
        return None, None, None, None

# Process low field datasets
lowfield_results = []
for ds in lowfield_datasets:
    result = analyze_rabi_frequency(ds, is_lowfield=True)
    if result[0] is not None:
        lowfield_results.append(result)

print(f"Processed {len(lowfield_results)} low field data points")


Processed 4 low field data points


## Save Processed Data


In [5]:
# Organize and save all processed data
processed_data = {
    'high_field': {
        'frequencies': all_frequencies,  # GHz
        'T2_values': all_T2_values,  # ns
        'T2_errors': all_dT2_values,  # ns
        'cv_stops': all_cv_stops,
        'distances': [(20 - cs) * 0.06 * 180 for cs in all_cv_stops]  # nm
    },
    'low_field': {
        'frequencies': [r[0] for r in lowfield_results],  # GHz
        'T2_values': [r[1] for r in lowfield_results],  # ns
        'f_errors': [r[2] for r in lowfield_results],  # GHz
        'T2_errors': [r[3] for r in lowfield_results]  # ns
    },
    'cv_stops': cv_stops
}

save_figure_data(
    processed_data,
    figure_number="fig5",
    filename="fig5c_driven_coherence.pkl",
    metadata={
        "description": "Driven spin coherence data for Figure 5c",
        "high_field_points": len(all_frequencies),
        "low_field_points": len(lowfield_results),
        "cv_stops": cv_stops
    }
)
print("✓ Saved processed data")


✓ Saved fig5 data to /Users/krzywdaja/Documents/spatial-correlations-conveyor/data_analysis/protection_code_repo/processed_data/fig5/fig5c_driven_coherence.pkl
  Metadata saved to /Users/krzywdaja/Documents/spatial-correlations-conveyor/data_analysis/protection_code_repo/processed_data/fig5/fig5c_driven_coherence.json
✓ Saved processed data


## Data for Fig6

In [10]:
# Extract specific T2 values for fig5_driven.pkl
# Load the processed data
import pickle
from pathlib import Path

processed_data_path = Path().resolve().parent / 'processed_data'
fig5c_file = processed_data_path / 'fig5' / 'fig5c_driven_coherence.pkl'

with open(fig5c_file, 'rb') as f:
    fig5c_data = pickle.load(f)

# Extract high field data
hf_freqs = np.array(fig5c_data['high_field']['frequencies'])  # GHz
hf_T2s = np.array(fig5c_data['high_field']['T2_values'])  # ns
hf_T2_errs = np.array(fig5c_data['high_field']['T2_errors'])  # ns
hf_cv_stops = np.array(fig5c_data['high_field']['cv_stops'])

# Extract low field data
lf_freqs = np.array(fig5c_data['low_field']['frequencies'])  # GHz
lf_T2s = np.array(fig5c_data['low_field']['T2_values'])  # ns
lf_T2_errs = np.array(fig5c_data['low_field']['T2_errors'])  # ns

# High field 100nm (cv_stop=15): find frequencies closest to 1 MHz and 1.5 MHz
cv_stop_100nm = 15
mask_100nm = hf_cv_stops == cv_stop_100nm
freqs_100nm = hf_freqs[mask_100nm]
T2s_100nm = hf_T2s[mask_100nm]
T2_errs_100nm = hf_T2_errs[mask_100nm]

# Find indices closest to target frequencies (in GHz)
target_1MHz = 0.001  # GHz
target_1_5MHz = 0.0017  # GHz

idx_1MHz = np.argmin(np.abs(freqs_100nm - target_1MHz))
idx_1_5MHz = np.argmin(np.abs(freqs_100nm - target_1_5MHz))

# Convert frequencies to MHz and extract T2 values
hf_100nm_data = [
    (freqs_100nm[idx_1MHz] * 1000, T2s_100nm[idx_1MHz], T2_errs_100nm[idx_1MHz]),  # 1 MHz
    (freqs_100nm[idx_1_5MHz] * 1000, T2s_100nm[idx_1_5MHz], T2_errs_100nm[idx_1_5MHz])  # 1.5 MHz
]

print(f"High field 50nm:")
print(f"  At {freqs_100nm[idx_1MHz]*1000:.3f} MHz: T2 = {T2s_100nm[idx_1MHz]:.1f} ± {T2_errs_100nm[idx_1MHz]:.1f} ns")
print(f"  At {freqs_100nm[idx_1_5MHz]*1000:.3f} MHz: T2 = {T2s_100nm[idx_1_5MHz]:.1f} ± {T2_errs_100nm[idx_1_5MHz]:.1f} ns")

# High field 200nm (cv_stop=5): use index 0 and index -2
cv_stop_200nm = 0
mask_200nm = hf_cv_stops == cv_stop_200nm
freqs_200nm = hf_freqs[mask_200nm]
T2s_200nm = hf_T2s[mask_200nm]
T2_errs_200nm = hf_T2_errs[mask_200nm]

# Sort by frequency to ensure consistent ordering
sort_idx = np.argsort(freqs_200nm)
freqs_200nm_sorted = freqs_200nm[sort_idx]
T2s_200nm_sorted = T2s_200nm[sort_idx]
T2_errs_200nm_sorted = T2_errs_200nm[sort_idx]

hf_200nm_data = [
    (freqs_200nm_sorted[0] * 1000, T2s_200nm_sorted[0], T2_errs_200nm_sorted[0]),  # index 0
    (freqs_200nm_sorted[-2] * 1000, T2s_200nm_sorted[-2], T2_errs_200nm_sorted[-2])  # index -2
]

print(f"\nHigh field 200nm:")
print(f"  Index 0 ({freqs_200nm_sorted[0]*1000:.3f} MHz): T2 = {T2s_200nm_sorted[0]:.1f} ± {T2_errs_200nm_sorted[0]:.1f} ns")
print(f"  Index -2 ({freqs_200nm_sorted[-2]*1000:.3f} MHz): T2 = {T2s_200nm_sorted[-2]:.1f} ± {T2_errs_200nm_sorted[-2]:.1f} ns")

# Low field: use index 1 and index -1
# Sort by frequency
sort_idx_lf = np.argsort(lf_freqs)
lf_freqs_sorted = lf_freqs[sort_idx_lf]
lf_T2s_sorted = lf_T2s[sort_idx_lf]
lf_T2_errs_sorted = lf_T2_errs[sort_idx_lf]

lf_200nm_data = [
    (lf_freqs_sorted[0] * 1000, lf_T2s_sorted[0], lf_T2_errs_sorted[0]),  # index 1
    (lf_freqs_sorted[-1] * 1000, lf_T2s_sorted[-1], lf_T2_errs_sorted[-1])  # index -1
]

print(f"\nLow field:")
print(f"  Index 1 ({lf_freqs_sorted[1]*1000:.3f} MHz): T2 = {lf_T2s_sorted[1]:.1f} ± {lf_T2_errs_sorted[1]:.1f} ns")
print(f"  Index -1 ({lf_freqs_sorted[-1]*1000:.3f} MHz): T2 = {lf_T2s_sorted[-1]:.1f} ± {lf_T2_errs_sorted[-1]:.1f} ns")

# Save to fig5_driven.pkl
driven_data = {
    'high': {
        100: hf_100nm_data,
        200: hf_200nm_data
    },
    'low': {
        200: lf_200nm_data
    }
}

fig5_driven_path = processed_data_path / 'fig5' / 'fig5_driven.pkl'
with open(fig5_driven_path, 'wb') as f:
    pickle.dump(driven_data, f)

print(f"\n✓ Saved driven data to {fig5_driven_path}")


High field 50nm:
  At 0.997 MHz: T2 = 3546.8 ± 254.0 ns
  At 1.640 MHz: T2 = 4388.6 ± 300.5 ns

High field 200nm:
  Index 0 (0.352 MHz): T2 = 8510.7 ± 1559.0 ns
  Index -2 (0.760 MHz): T2 = 12245.9 ± 873.7 ns

Low field:
  Index 1 (0.677 MHz): T2 = 12548.1 ± 1833.9 ns
  Index -1 (1.217 MHz): T2 = 21098.7 ± 2754.8 ns

✓ Saved driven data to /Users/krzywdaja/Documents/spatial-correlations-conveyor/data_analysis/protection_code_repo/processed_data/fig5/fig5_driven.pkl
